# Combining Tianshou's PPO with our CorePolicy

This notebook is mostly meant as a test, and a showcase of how we can combine a CorePolicy with policies provided by Tianshou.

In [1]:
import tianshou as ts 
from tianshou.utils import TensorboardLogger

import torch
from torch.utils.tensorboard import SummaryWriter

import os
from datetime import datetime

from environments import Resetting
from networks import NetHackObsNet, GoalNetHackActor, GoalNetHackCritic
from policies import PPOBasedPolicy
from models import SelfModel, EnvModel
from intrinsic import ICM
from core import GoalCollector, GoalVectorReplayBuffer, GoalOnpolicyTrainer

from nle.env.tasks import NetHackGold

## Setup

In [2]:
env = Resetting(NetHackGold())
device = torch.device("cpu")

In [3]:
num_train_envs = 2
num_test_envs = 1

train_envs = ts.env.DummyVectorEnv([lambda: env for _ in range(num_train_envs)])
test_envs = ts.env.DummyVectorEnv([lambda: env for _ in range(num_test_envs)])

In [4]:
obs_net = NetHackObsNet(env.observation_space, device=device)

actor_net = GoalNetHackActor(obs_net, env.action_space, device=device)
critic_net = GoalNetHackCritic(obs_net, device=device)

In [5]:
train_buf = GoalVectorReplayBuffer(50, num_train_envs)
test_buf = GoalVectorReplayBuffer(1, num_train_envs)

env_model = EnvModel() # this does nothing, for the moment
self_model = SelfModel(obs_net, env.action_space, train_buf, ICM, her_horizon=3, device=device)

In [6]:
# using a single optimizer for actor and critic simplifies the training loop and is more computationally efficient
# BUT gradient updates in one network will influence the gradient updates in the other, and this might create unexpected problems...
combined_params = set(list(actor_net.parameters()) + list(critic_net.parameters()))
optimizer = torch.optim.Adam(combined_params, lr=3e-4)

## Custom PPO-based policy

In [7]:
policy = PPOBasedPolicy(
    self_model=self_model,
    env_model=env_model,
    act_net=actor_net, 
    critic_net=critic_net, 
    optim=optimizer,
    action_space=env.action_space,
    observation_space=env.observation_space,
    device=device
)

In [8]:
train_collector = GoalCollector(policy, train_envs, train_buf)
test_collector = GoalCollector(policy, test_envs, test_buf)

In [9]:
num_epochs = 1
num_steps_per_epoch = 10

step_per_collect = 10
episode_per_test = 1
batch_size = 5

timestamp = datetime.now().strftime("%d%m%Y-%H%M%S")
log_path = os.path.join("../logs", "ppo_based", timestamp)
writer = SummaryWriter(log_path)
logger = TensorboardLogger(writer)

In [10]:
trainer = GoalOnpolicyTrainer(
    policy=policy, 
    train_collector=train_collector, 
    test_collector=test_collector,
    repeat_per_collect=1,
    max_epoch=num_epochs,
    step_per_epoch=num_steps_per_epoch,
    step_per_collect=step_per_collect,
    episode_per_test=episode_per_test,
    batch_size=batch_size,
    logger=logger,
    device=device
)

In [11]:
# TODO HER doesn't contribute to the intrinsic rewards displayed here => why?
for epoch_stats in trainer:
    # TODO a more informative print, plots, logging, etc.
    print("\nNothing to see here. Move along, now...")

KeyboardInterrupt: 